# crawl alle adviezen van adviescollegeicttoetsing.nl

* <https://www.adviescollegeicttoetsing.nl/onderzoeken/afgeronde-onderzoeken-2023-en-eerder>
* marx
* 2024-03-30, aangepast van de Raadvan State crwaler


## V1

 

Volgens  schema van Maik.


## Volledigheid crawl

Op 30 Maart 2024 hebben we de website <https://www.adviescollegeicttoetsing.nl/onderzoeken> van het Adviescollege icttoetsing gecrawled. We hebben zowel de pagina met afgeronde onderzoeken uit 2024 als de pagina met voorgaande onderzoeken bezocht, de tabellen eruit gehaald en aan elkaar geplkt tot een tabel met 121 adviezen. Die hebben we omgezet naar ons schema. We hebben gezorgd dat de PDF van het advies, de HTML van de pagina van het advies en alle hyperlinks naar externe stukken ook opgeslagen werden, en zodoende ook geindexeeerd. Heel vaak is er 1 externe link, naar de reactie van de Minister.

Voor elk advies hebben we de volgende data verzameld: 'dc_title', 'foi_decisionDate', 'foi_publishedDate', 'dc_description',
       'dc_publisher', 'dc_type', 'foi_files' (hierin de urls naar de bovengenoemde files).
 

###  dagelijkse crawler  

*  Dit script kan als dagelijkse crawler gebruikt worden.
* Alleen moet in 2025 de urls even aangepast worden.
* Het script haalt steeds alles op. 

In [1]:

import re
from bs4 import BeautifulSoup
from lxml import etree
import pandas as pd
import json
import requests
from requests_html import HTMLSession
from requests_html import HTML, AsyncHTMLSession
from time import sleep 
from datetime import datetime
today=  datetime.today().strftime('%Y-%m-%d')

def xpadje(element,path_expression):
    if element.xpath(path_expression):
        return element.xpath(path_expression)[0]
    else:
        
        return ''
    
## Deze adviezen bevatten geen tekst en worden niet gepubliceerd en willen we dus niet meenemen in Woogle.
## Zijn er nog heel wat hoor.

def ongepubliceerd_advies(adviestekst):
    check='Dit advies is een zogenoemd advies conform'
    return check in adviestekst



In [2]:
DC_TYPE = '2e'  # adviezen
DC_PUBLISHER='oorg10165'
DC_CREATOR='https://www.adviescollegeicttoetsing.nl'
BASE_URL='https://www.adviescollegeicttoetsing.nl'
RETRIEVED_DATE=today

headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}

### specifiek voor crawl
LAST_PAGE=1210
PDF_Suffix=  '?export=pdf&tab=volledigetekst'

# step 1 crawl all pages with links to woo dossiers

* Dat gaat hier wat nicer want we hebben mooie tabellen, waar we prachtig spul zo met pandas uit kunnen halen.
* Leuk!

In [3]:
url='https://www.adviescollegeicttoetsing.nl/onderzoeken/afgeronde-onderzoeken-2023-en-eerder'
url2='https://www.adviescollegeicttoetsing.nl/onderzoeken/afgeronde-onderzoeken-2024'
dfs=pd.read_html(url, extract_links='body')
dfs2=pd.read_html(url2, extract_links='body')

df= pd.concat(dfs+dfs2)
df.shape

(121, 4)

In [4]:
# even opschonen

for c in ['Project / advies']: # df.columns:
    df[c+'url']=df[c].str.get(1)
    df[c]=df[c].str.get(0)
for c in set(df.columns) - {'Project / advies','Project / adviesurl'}: # df.columns:
     
    df[c]=df[c].str.get(0)
df['Project / adviesurl']= BASE_URL+df['Project / adviesurl']
print(df.shape)
dfD=df.to_dict(orient='records')

(121, 5)


In [5]:
dfD[8]

{'Ministerie': 'Binnenlandse Zaken en Koninkrijksrelaties',
 'Project / advies': 'Toekomst BRP',
 'Omschrijving': 'Ervoor zorgen dat de Basisregistratie Personen (BRP) wordt vernieuwd door het bestaande systeem door te ontwikkelen.',
 'Datum\xa0advies': '19-06-2023',
 'Project / adviesurl': 'https://www.adviescollegeicttoetsing.nl/onderzoeken/documenten/publicaties/2023/06/19/bit-advies-toekomst-brp'}

# step 2 extract links to woo dosiers

* dat is hierboven meteen gedaan

# step 3 extract infor for each woo dossier


## schema van Maik

```
{
    "dc_source": "https://www.raadvanstate.nl/adviezen/@140896/w12-23-00375-iii/",
    "dc_title": "Wet implementatie EU-richtlijn toereikende minimumlonen.",
    "foi_publishedDate": "2024-03-18",
    "foi_decisionDate": "2023-12-18",
    "dc_description": "Bij Kabinetsmissive van 18 december 2023, no.2023002985, heeft Uwe Majesteit, op voordracht van de Minister van Sociale Zaken en Werkgelegenheid, bij de Afdeling advisering van de Raad van State ter overweging aanhangig gemaakt het voorstel van wet tot wijziging van de Wet minimumloon en minimumvakantiebijslag in verband met de implementatie van Richtlijn (EU) 2022/2041 van het Europees Parlement en de Raad van 19 oktober 2022 betreffende toereikende minimumlonen in de Europese Unie (Pb EU 2022, L 275) (Wet implementatie EU-richtlijn toereikende minimumlonen), met memorie van toelichting.",
    "dc_publisher": "oorg10008",
    "dc_type": "2e",
    "foi_files": {
        "foi_documenten": [
            {
                "dc_title": "W12-23-00375-III",
                "dc_source": "https://www.raadvanstate.nl/adviezen/@140896/w12-23-00375-iii/?export=pdf&tab=volledigetekst",
                "foi_url_on_web": true
            }
        ]
    }
}
```

* Als beschrijving de eerste alinea van de tekst
* Geen bodytext, maar een pdf document met de tekst
* Ik ben nog niet helemaal uit over de datums, op de site staat in ieder geval een publicatiedatum, maar ik weet niet of 'datum aanhangig' echt valt onder besluitdatum die we wel vaker gebruiken. Daarnaast is er ook nog een adviesdatum die we nog niet dekken


```
{'Ministerie': 'Binnenlandse Zaken en Koninkrijksrelaties',
 'Project / advies': 'Toekomst BRP',
 'Omschrijving': 'Ervoor zorgen dat de Basisregistratie Personen (BRP) wordt vernieuwd door het bestaande systeem door te ontwikkelen.',
 'Datum\xa0advies': '19-06-2023',
 'Project / adviesurl': 'https://www.adviescollegeicttoetsing.nl/onderzoeken/documenten/publicaties/2023/06/19/bit-advies-toekomst-brp'}
``` 

In [6]:
def parse_one_advies(d):
    url= d['Project / adviesurl']
    dutchdate=d['Datum\xa0advies']
    
    r = requests.get(url,headers= headers)
    dom = etree.HTML(r.content)
    woo=dict()
    # meta property="og:url" content=
    woo['dc_source']=dom.xpath('.//meta[@property="og:url"]/@content')[0]
    # meta name="DCTERMS.title" content=
    woo['dc_title']=dom.xpath('.//meta[@name="DCTERMS.title"]/@content')[0] 
    # <meta name="DCTERMS.available" title="DCTERMS.Period" content="start=2022-12-22;"/>
    
    woo['foi_decisionDate']= '-'.join(dutchdate.split('-')[::-1])
    woo['foi_publishedDate']= dom.xpath('.//meta[@name="DCTERMS.available"]/@content')[0].replace('start=','')
    woo['dc_description']= 'Project / advies ' + d['Project / advies']+'. ' + ' Ministerie '+ d['Ministerie'] +'. ' +d['Omschrijving']
    
    
    # als er een samenvattng is wordt dat de description, anders de eerste alinea
     
    woo['dc_publisher']=DC_PUBLISHER
    woo["dc_type"]= DC_TYPE
    #woo['foi_retrievedDate']=RETRIEVED_DATE
    #woo['foi_bodyTextOriginal']= '\n'.join(dom.xpath('.//div[@id="volledigetekst"]//*[self::p or self::div]//text()') )
    
     
      
    advies={"dc_title":  xpadje(dom,'//h1[@class="download"]//text()').strip(),
                "dc_source":  BASE_URL+xpadje(dom,'//a[@class="download-chunk pdf"]/@href'),
                "foi_url_on_web": True}
    html_page={"dc_title":   woo['dc_title'],
                "dc_source":   woo["dc_source"],
                "foi_url_on_web": True}
    links= dom.xpath('//a[@class="external"]')
    
    rest=[{
                "dc_title": xpadje(l,'./text()'),
                "dc_source": xpadje(l,'@href'),
                "foi_url_on_web": True
            } for l in links]
    woo["foi_files"]= {
        "foi_documenten": [advies,html_page]+rest
    }
    return woo

parse_one_advies(dfD[9]) 

{'dc_source': 'https://www.adviescollegeicttoetsing.nl/documenten/publicaties/2023/05/12/bit-advies-programma-vrachtwagenheffing',
 'dc_title': 'BIT-advies programma Vrachtwagenheffing - Publicatie - Adviescollege ICT-toetsing',
 'foi_decisionDate': '2023-05-12',
 'foi_publishedDate': '2023-05-12;',
 'dc_description': 'Project / advies Vrachtwagenheffing.  Ministerie Infrastructuur en Waterstaat. Introductie heffing waarmee het vrachtverkeer voor gereden kilometers gaat betalen.',
 'dc_publisher': 'oorg10165',
 'dc_type': '2e',
 'foi_files': {'foi_documenten': [{'dc_title': 'BIT-advies programma Vrachtwagenheffing',
    'dc_source': 'https://www.adviescollegeicttoetsing.nl/binaries/adviescollegeicttoetsing/documenten/publicaties/2023/05/12/bit-advies-programma-vrachtwagenheffing/BIT-advies+programma+Vrachtwagenheffing.pdf',
    'foi_url_on_web': True},
   {'dc_title': 'BIT-advies programma Vrachtwagenheffing - Publicatie - Adviescollege ICT-toetsing',
    'dc_source': 'https://www.advi

In [7]:
%%time
Dx=dict()

for c,l in enumerate(dfD):
     
    try:
        o= parse_one_advies(l)
        Dx[c]=o
         
    except Exception as e:
        print('WRONG',url)
        print(e)

len(Dx)

CPU times: user 6.77 s, sys: 267 ms, total: 7.03 s
Wall time: 21.6 s


121

# step 4 store as dataframe

In [9]:
df =pd.DataFrame.from_dict(Dx, orient='index')
#df.columns=['dc_source','foi_decisionDate','dc:title','dc:description','files']
#df['dc_source']=df.dossier_url
#df.drop('dossier_url',1, inplace=True)
#df['foi_retrievedDate']=RETRIEVED_DATE
df.index=df.dc_source
df.drop('dc_source',axis=1,inplace=True)
print(df.shape)
df.head()

(121, 7)


,dc_title,foi_decisionDate,foi_publishedDate,dc_description,dc_publisher,dc_type,foi_files
dc_source,,,,,,,
https://www.adviescollegeicttoetsing.nl/documenten/publicaties/2023/12/22/bit-advies-de-instandhouding-transitie,BIT-advies De Instandhouding Transitie - Publi...,2023-12-22,2023-12-22;,Project / advies De Instandhouding Transitie. ...,oorg10165,2e,{'foi_documenten': [{'dc_title': 'BIT-advies D...
https://www.adviescollegeicttoetsing.nl/documenten/publicaties/2023/12/22/bit-advies-iv-vernieuwingstraject-bij-de-ind,Advies IV vernieuwingstraject bij de IND - Pub...,2023-12-22,2023-12-22;,Project / advies IV vernieuwingstraject bij de...,oorg10165,2e,{'foi_documenten': [{'dc_title': 'Advies IV ve...
https://www.adviescollegeicttoetsing.nl/documenten/publicaties/2023/12/21/bit-advies-programma-openvms,BIT-advies Programma OpenVMS - Publicatie - Ad...,2023-12-21,2023-12-21;,Project / advies Programma OpenVMS. Ministeri...,oorg10165,2e,{'foi_documenten': [{'dc_title': 'BIT-advies P...
https://www.adviescollegeicttoetsing.nl/documenten/publicaties/2023/10/09/advies-toekomstbestendigheid-van-missiekritische-communicatiesystemen-noova,Advies toekomstbestendigheid van missiekritisc...,2023-10-09,2023-10-09;,Project / advies Nieuw Openbare Orde en Veilig...,oorg10165,2e,{'foi_documenten': [{'dc_title': 'Advies toeko...
https://www.adviescollegeicttoetsing.nl/documenten/publicaties/2023/09/25/bit-advies-doorontwikkelen-applicatielandschap-bekostiging-2,BIT-advies Doorontwikkelen Applicatielandschap...,2023-09-25,2023-09-25;,Project / advies Doorontwikkelen Applicatielan...,oorg10165,2e,{'foi_documenten': [{'dc_title': 'BIT-advies D...


In [10]:
f='BITcollege'+today+'.json.gz'
df.to_json(f,orient='records', indent=3 )  #)
!ls -lh $f

-rw-r--r--  1 admin  staff    19K Mar 30 10:47 BITcollege2024-03-30.json.gz
